<a href="https://colab.research.google.com/github/pranavsrinivas29/Pyspark/blob/main/Pyspark3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handling Missing Values

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=551eafcd4c7625d0821997dd7a3dcdcd740bbfc776f63c5e81e3a64d3c251b9e
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('DataFrame').getOrCreate()
df_pyspark = spark.read.option('header','true').csv('/content/drive/MyDrive/Book1.csv',sep=';',inferSchema=True)

In [23]:
df_pyspark.show()

+------+----+------+----------+
|  Name| Age|Salary|Experience|
+------+----+------+----------+
|  John|  23| 34000|         8|
| Smith|  21| 56000|      NULL|
| Steve|  43|  NULL|         2|
|  Mark|  25| 23990|      NULL|
| Starc|NULL| 45000|         6|
|  NULL|  34|  NULL|         2|
|Austin|  34| 12000|      NULL|
+------+----+------+----------+



## Dropping the cols

In [24]:
df_pyspark.drop('Age').show()

+------+------+----------+
|  Name|Salary|Experience|
+------+------+----------+
|  John| 34000|         8|
| Smith| 56000|      NULL|
| Steve|  NULL|         2|
|  Mark| 23990|      NULL|
| Starc| 45000|         6|
|  NULL|  NULL|         2|
|Austin| 12000|      NULL|
+------+------+----------+



In [25]:
df_pyspark.na.drop().show()

+----+---+------+----------+
|Name|Age|Salary|Experience|
+----+---+------+----------+
|John| 23| 34000|         8|
+----+---+------+----------+



In [26]:
#Can use all if all values in row in null and want to drop it
df_pyspark.na.drop(how="any").show()

+----+---+------+----------+
|Name|Age|Salary|Experience|
+----+---+------+----------+
|John| 23| 34000|         8|
+----+---+------+----------+



In [29]:
#shud hav atleast 2 non null values
df_pyspark.na.drop(how="any",thresh=2).show()

+------+----+------+----------+
|  Name| Age|Salary|Experience|
+------+----+------+----------+
|  John|  23| 34000|         8|
| Smith|  21| 56000|      NULL|
| Steve|  43|  NULL|         2|
|  Mark|  25| 23990|      NULL|
| Starc|NULL| 45000|         6|
|  NULL|  34|  NULL|         2|
|Austin|  34| 12000|      NULL|
+------+----+------+----------+



In [30]:
#drops null considering only experience
df_pyspark.na.drop(how="any",subset=['Experience']).show()

+-----+----+------+----------+
| Name| Age|Salary|Experience|
+-----+----+------+----------+
| John|  23| 34000|         8|
|Steve|  43|  NULL|         2|
|Starc|NULL| 45000|         6|
| NULL|  34|  NULL|         2|
+-----+----+------+----------+



## Filling Values

In [38]:
df_pyspark.na.fill('Missing Values').show()

+--------------+----+------+----------+
|          Name| Age|Salary|Experience|
+--------------+----+------+----------+
|          John|  23| 34000|         8|
|         Smith|  21| 56000|      NULL|
|         Steve|  43|  NULL|         2|
|          Mark|  25| 23990|      NULL|
|         Starc|NULL| 45000|         6|
|Missing Values|  34|  NULL|         2|
|        Austin|  34| 12000|      NULL|
+--------------+----+------+----------+



## Imputing Values

In [46]:
from pyspark.ml.feature import Imputer
imputer=Imputer(
    inputCols=['Age','Salary','Experience'],
    outputCols=["{}_mean_imputed".format(c) for c in ['Age','Salary','Experience']]
).setStrategy("mean")

In [47]:
imputer.fit(df_pyspark).transform(df_pyspark).show()

+------+----+------+----------+----------------+-------------------+-----------------------+
|  Name| Age|Salary|Experience|Age_mean_imputed|Salary_mean_imputed|Experience_mean_imputed|
+------+----+------+----------+----------------+-------------------+-----------------------+
|  John|  23| 34000|         8|              23|              34000|                      8|
| Smith|  21| 56000|      NULL|              21|              56000|                      4|
| Steve|  43|  NULL|         2|              43|              34198|                      2|
|  Mark|  25| 23990|      NULL|              25|              23990|                      4|
| Starc|NULL| 45000|         6|              30|              45000|                      6|
|  NULL|  34|  NULL|         2|              34|              34198|                      2|
|Austin|  34| 12000|      NULL|              34|              12000|                      4|
+------+----+------+----------+----------------+-------------------+--

In [48]:
from pyspark.ml.feature import Imputer
imputer=Imputer(
    inputCols=['Age','Salary','Experience'],
    outputCols=["{}_median_imputed".format(c) for c in ['Age','Salary','Experience']]
).setStrategy("median")

imputer.fit(df_pyspark).transform(df_pyspark).show()

+------+----+------+----------+------------------+---------------------+-------------------------+
|  Name| Age|Salary|Experience|Age_median_imputed|Salary_median_imputed|Experience_median_imputed|
+------+----+------+----------+------------------+---------------------+-------------------------+
|  John|  23| 34000|         8|                23|                34000|                        8|
| Smith|  21| 56000|      NULL|                21|                56000|                        2|
| Steve|  43|  NULL|         2|                43|                34000|                        2|
|  Mark|  25| 23990|      NULL|                25|                23990|                        2|
| Starc|NULL| 45000|         6|                25|                45000|                        6|
|  NULL|  34|  NULL|         2|                34|                34000|                        2|
|Austin|  34| 12000|      NULL|                34|                12000|                        2|
+------+--